In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px

### Compute ressources

In [2]:
# Ncores
date_compute_inital = "2027-01-01" 
N_nodes = 24
cores_per_node = 64
N_cores = N_nodes * cores_per_node
print(f"Total number of cores available Y1: {N_cores}")

date_compute_increase = "2028-01-01" 
node_increase_factor = 2
N_nodes_Y2 = int(N_nodes * node_increase_factor)
N_cores_Y2 = N_nodes_Y2 * cores_per_node
print(f"Total number of cores available Y2: {N_cores_Y2}")

Total number of cores available Y1: 1536
Total number of cores available Y2: 3072


### Processing time in deg2 per core hours

In [3]:
# IMCOM
# Time per filter per deg2 in hours
imcom_time = 11_000 / 1.17

# Metadetection
# Time per deg2 in hours
mdet_time = 100

### Survey area in deg2

In [4]:
# Deep field
deep_field_area = 20
deep_field_n_filters = 3
obs_end_date = "2027-02-10"  # YYYY-MM-DD

# Medium tier Y1
medium_tier_area = 200
medium_tier_n_filters = 3

# Medium tier Y2
obs_medium_end_date = "2028-12-31"  # YYYY-MM-DD
medium_tier_area_Y2 = 1000

### Analysis Planning

In [5]:
# We will likely do a cycle of processing and analysis before doing the processing of the medium tier
analysis_time = 30  # days
n_cycles = 3

### Timeline

In [6]:
tasks_list = []
# Y1
for i in range(n_cycles):
    if i == 0:
        deps = None
    else:
        deps = f"Diagnostic_{i}"
    task = {
        "task": "Processing Deep Field",
        "task_id": f"Processing_{i+1}",
        "duration_days": (imcom_time * deep_field_n_filters * deep_field_area + mdet_time * deep_field_area) / N_cores / 24,
        "depends_on": deps,
        "start": None,
        "end": None,
    }
    tasks_list.append(task)
    task = {
        "task": "Diagnostic",
        "task_id": f"Diagnostic_{i+1}",
        "duration_days": analysis_time,
        "depends_on": f"Processing_{i+1}",
        "start": None,
        "end": None,
    }
    tasks_list.append(task)
task = {
    "task": "Processing Medium Tier 200deg2",
    "task_id": "Processing_Medium_Tier_200deg2",
    "duration_days": (imcom_time * medium_tier_n_filters * medium_tier_area + mdet_time * medium_tier_area) / N_cores / 24,
    "depends_on": f"Diagnostic_{n_cycles}",
    "start": None,
    "end": None,
}
tasks_list.append(task)
task = {
    "task": "Diagnostic",
    "task_id": "Diagnostic_Y1",
    "duration_days": 2*30,
    "depends_on": "Processing_Medium_Tier_200deg2",
    "start": None,
    "end": None,
}
tasks_list.append(task)

task = {
    "task": "Initial Science on Deep Field",
    "task_id": "Initial Science on Deep Field",
    "duration_days": 6 * 30,
    "depends_on": f"Diagnostic_{n_cycles}",
    "start": None,
    "end": None,
}
tasks_list.append(task)


# Y2
task = {
    "task": "Processing Medium Tier 1000deg2",
    "task_id": "Processing_Medium_Tier 1000deg2",
    "duration_days": (imcom_time * medium_tier_n_filters * medium_tier_area_Y2 + mdet_time * medium_tier_area_Y2) / N_cores_Y2 / 24,
    "depends_on": f"Diagnostic_Y1",
    "start": None,
    "end": None,
}
tasks_list.append(task)
task = {
    "task": "Y1 Science",
    "task_id": "Y1_Science",
    "duration_days": 12 * 30,
    "depends_on": f"Diagnostic_Y1",
    "start": None,
    "end": None,
}
tasks_list.append(task)

df = pd.DataFrame(tasks_list)

In [7]:
df

,task,task_id,duration_days,depends_on,start,end
0,Processing Deep Field,Processing_1,15.356515,None,None,None
1,Diagnostic,Diagnostic_1,30.000000,Processing_1,None,None
2,Processing Deep Field,Processing_2,15.356515,Diagnostic_1,None,None
3,Diagnostic,Diagnostic_2,30.000000,Processing_2,None,None
4,Processing Deep Field,Processing_3,15.356515,Diagnostic_2,None,None
5,Diagnostic,Diagnostic_3,30.000000,Processing_3,None,None
6,Processing Medium Tier 200deg2,Processing_Medium_Tier_200deg2,153.565149,Diagnostic_3,None,None
7,Diagnostic,Diagnostic_Y1,60.000000,Processing_Medium_Tier_200deg2,None,None
8,Initial Science on Deep Field,Initial Science on Deep Field,180.000000,Diagnostic_3,None,None
9,Processing Medium Tier 1000deg2,Processing_Medium_Tier 1000deg2,383.912872,Diagnostic_Y1,None,None


In [8]:
start_date = datetime.strptime(obs_end_date, "%Y-%m-%d")

# Compute start/end dates
task_start = {}
task_end = {}

for _, row in df.iterrows():
    dep = row["depends_on"]
    if dep:
        start_time = task_end[dep]
    else:
        start_time = start_date  # independent task
    end_time = start_time + timedelta(days=row["duration_days"])
    
    task_start[row["task_id"]] = start_time
    task_end[row["task_id"]] = end_time


df["start"] = df["task_id"].map(task_start)
df["end"] = df["task_id"].map(task_end)

In [9]:
df

,task,task_id,duration_days,depends_on,start,end
0,Processing Deep Field,Processing_1,15.356515,None,2027-02-10 00:00:00.000000,2027-02-25 08:33:22.884615
1,Diagnostic,Diagnostic_1,30.000000,Processing_1,2027-02-25 08:33:22.884615,2027-03-27 08:33:22.884615
2,Processing Deep Field,Processing_2,15.356515,Diagnostic_1,2027-03-27 08:33:22.884615,2027-04-11 17:06:45.769230
3,Diagnostic,Diagnostic_2,30.000000,Processing_2,2027-04-11 17:06:45.769230,2027-05-11 17:06:45.769230
4,Processing Deep Field,Processing_3,15.356515,Diagnostic_2,2027-05-11 17:06:45.769230,2027-05-27 01:40:08.653845
5,Diagnostic,Diagnostic_3,30.000000,Processing_3,2027-05-27 01:40:08.653845,2027-06-26 01:40:08.653845
6,Processing Medium Tier 200deg2,Processing_Medium_Tier_200deg2,153.565149,Diagnostic_3,2027-06-26 01:40:08.653845,2027-11-26 15:13:57.499999
7,Diagnostic,Diagnostic_Y1,60.000000,Processing_Medium_Tier_200deg2,2027-11-26 15:13:57.499999,2028-01-25 15:13:57.499999
8,Initial Science on Deep Field,Initial Science on Deep Field,180.000000,Diagnostic_3,2027-06-26 01:40:08.653845,2027-12-23 01:40:08.653845
9,Processing Medium Tier 1000deg2,Processing_Medium_Tier 1000deg2,383.912872,Diagnostic_Y1,2028-01-25 15:13:57.499999,2029-02-12 13:08:29.615384


In [10]:
fig = px.timeline(df, x_start="start", x_end="end", y="task", color="task", title="Project Timeline")
# fig.update_yaxes(autorange="reversed")  # Gantt-style

# LSST DP2
date_DESC_DP2_obs = "2026-12-1"
fig.add_shape(
    type="line",
    x0=date_DESC_DP2_obs,
    x1=date_DESC_DP2_obs,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="blue", width=2, dash="dash")
)
fig.add_annotation(
    x=date_DESC_DP2_obs,
    y=1,
    yref="paper",
    text=f"LSST DP2 (~07/2026)",
    showarrow=False,
    yshift=50,
    font=dict(color="blue")
)

# LSST DR1
date_DESC_DP2_obs = "2028-03-01"
fig.add_shape(
    type="line",
    x0=date_DESC_DP2_obs,
    x1=date_DESC_DP2_obs,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="blue", width=2, dash="dash")
)
fig.add_annotation(
    x=date_DESC_DP2_obs,
    y=1,
    yref="paper",
    text=f"LSST DR1",
    showarrow=False,
    yshift=50,
    font=dict(color="blue")
)

# Initial compute ressources
fig.add_shape(
    type="line",
    x0=date_compute_inital,
    x1=date_compute_inital,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2)
)
fig.add_annotation(
    x=date_compute_inital,
    y=1,
    yref="paper",
    text=f"{N_nodes} nodes, {N_cores} cores",
    showarrow=False,
    yshift=20,
    font=dict(color="black")
)

# Increse of compute ressources
fig.add_shape(
    type="line",
    x0=date_compute_increase,
    x1=date_compute_increase,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2)
)
fig.add_annotation(
    x=date_compute_increase,
    y=1,
    yref="paper",
    text=f"{N_nodes_Y2} nodes, {N_cores_Y2} cores",
    showarrow=False,
    yshift=20,
    font=dict(color="black")
)

# SOC observations Deep
fig.add_shape(
    type="line",
    x0=obs_end_date,
    x1=obs_end_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="red", width=2, dash="dash")
)
fig.add_annotation(
    x=obs_end_date,
    y=1,
    yref="paper",
    text=f"Deep field Obs. end",
    showarrow=False,
    yshift=35,
    font=dict(color="red")
)

# SOC observations Medium
fig.add_shape(
    type="line",
    x0=obs_medium_end_date,
    x1=obs_medium_end_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="red", width=2, dash="dash")
)
fig.add_annotation(
    x=obs_medium_end_date,
    y=1,
    yref="paper",
    text="Medium Tier field 1 + Some field 2 Obs. end (<1000deg2)",
    showarrow=False,
    yshift=35,
    font=dict(color="red")
)

fig.show()

In [11]:
print(
    "Summary\n"
    "=======\n"
    "Y1\n"
    "--\n"
    f"N cores: {N_cores}\n"
    "\n"
    f"Deep field area: {deep_field_area} deg2\n"
    f"N filters: {deep_field_n_filters}\n"
    f"Deep field Processing start date: {task_start['Processing_1'].strftime('%Y-%m-%d')}\n"
    f"Deep field Processing time: {round((task_end['Processing_1'] - task_start['Processing_1']).total_seconds() / 3600 / 24, 2)} days\n"
    "\n"
    f"Medium tier area: {medium_tier_area} deg2\n"
    f"N filters: {medium_tier_n_filters}\n"
    f"Medium Processing start date: {task_start['Processing_Medium_Tier_200deg2'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing end date: {task_end['Processing_Medium_Tier_200deg2'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing duration: {round((task_end['Processing_Medium_Tier_200deg2'] - task_start['Processing_Medium_Tier_200deg2']).total_seconds() / 3600 / 24, 2)} days\n"
    "\n\n"
    "Y2\n"
    "--\n"
    f"N cores: {N_cores_Y2}\n"
    "\n"
    f"Medium tier area: {medium_tier_area_Y2} deg2\n"
    f"Including reprocessing of Y1 data\n"
    f"N filters: {medium_tier_n_filters}\n"
    f"Medium Processing start date: {task_start['Processing_Medium_Tier 1000deg2'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing end date: {task_end['Processing_Medium_Tier 1000deg2'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing duration: {round((task_end['Processing_Medium_Tier 1000deg2'] - task_start['Processing_Medium_Tier 1000deg2']).total_seconds() / 3600 / 24, 2)} days\n"
)

Summary
Y1
--
N cores: 1536

Deep field area: 20 deg2
N filters: 3
Deep field Processing start date: 2027-02-10
Deep field Processing time: 15.36 days

Medium tier area: 200 deg2
N filters: 3
Medium Processing start date: 2027-06-26
Medium Processing end date: 2027-11-26
Medium Processing duration: 153.57 days


Y2
--
N cores: 3072

Medium tier area: 1000 deg2
Including reprocessing of Y1 data
N filters: 3
Medium Processing start date: 2028-01-25
Medium Processing end date: 2029-02-12
Medium Processing duration: 383.91 days

